In [6]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("chinook.db")

In [2]:
#1. Find the total amount spent by each customer on purchases (considering invoices).
query = """
SELECT 
    c.CustomerId,
    c.FirstName || ' ' || c.LastName AS CustomerName,
    SUM(i.Total) AS TotalSpent
FROM 
    customers c
JOIN 
    invoices i ON c.CustomerId = i.CustomerId
GROUP BY 
    c.CustomerId
ORDER BY 
    TotalSpent DESC;
"""
total_spent_df = pd.read_sql_query(query, conn)

print(total_spent_df)

    CustomerId           CustomerName  TotalSpent
0            6            Helena Holý       49.62
1           26     Richard Cunningham       47.62
2           57             Luis Rojas       46.62
3           45        Ladislav Kovács       45.62
4           46          Hugh O'Reilly       45.62
5           28          Julia Barnett       43.62
6           24          Frank Ralston       43.62
7           37        Fynn Zimmermann       43.62
8            7          Astrid Gruber       42.62
9           25         Victor Stevens       42.62
10          44       Terhi Hämäläinen       41.62
11           5  František Wichterlová       40.62
12          43       Isabelle Mercier       40.62
13          48  Johannes Van der Berg       40.62
14          17             Jack Smith       39.62
15          34         João Fernandes       39.62
16           1         Luís Gonçalves       39.62
17           3      François Tremblay       39.62
18           4           Bjørn Hansen       39.62


In [3]:
#2. Identify the top 5 customers with the highest total purchase amounts.
query = """
SELECT 
    c.CustomerId,
    c.FirstName || ' ' || c.LastName AS CustomerName,
    SUM(i.Total) AS TotalSpent
FROM 
    customers c
JOIN 
    invoices i ON c.CustomerId = i.CustomerId
GROUP BY 
    c.CustomerId
ORDER BY 
    TotalSpent DESC
LIMIT 5;
"""
top_customers = pd.read_sql_query(query, conn)
print(top_customers)

   CustomerId        CustomerName  TotalSpent
0           6         Helena Holý       49.62
1          26  Richard Cunningham       47.62
2          57          Luis Rojas       46.62
3          45     Ladislav Kovács       45.62
4          46       Hugh O'Reilly       45.62


In [4]:
#3. Display the customer ID, name, and the total amount spent for the top 5 customers.
query = """
SELECT 
    c.CustomerId,
    c.FirstName || ' ' || c.LastName AS CustomerName,
    SUM(i.Total) AS TotalSpent
FROM 
    customers c
JOIN 
    invoices i ON c.CustomerId = i.CustomerId
GROUP BY 
    c.CustomerId
ORDER BY 
    TotalSpent DESC
LIMIT 5;
"""

top_5_customers = pd.read_sql_query(query, conn)
print(top_5_customers)

   CustomerId        CustomerName  TotalSpent
0           6         Helena Holý       49.62
1          26  Richard Cunningham       47.62
2          57          Luis Rojas       46.62
3          45     Ladislav Kovács       45.62
4          46       Hugh O'Reilly       45.62


In [7]:
#4. Determine the percentage of customers who prefer to buy individual tracks instead of full albums.
invoice_tracks_query = """
SELECT 
    i.CustomerId,
    il.TrackId,
    t.AlbumId
FROM 
    invoice_items il
JOIN 
    invoices i ON il.InvoiceId = i.InvoiceId
JOIN 
    tracks t ON il.TrackId = t.TrackId;
"""
invoice_tracks = pd.read_sql_query(invoice_tracks_query, conn)

album_tracks_query = """
SELECT 
    AlbumId,
    COUNT(*) AS TotalTracks
FROM 
    tracks
GROUP BY 
    AlbumId;
"""
album_tracks = pd.read_sql_query(album_tracks_query, conn)

merged = invoice_tracks.merge(album_tracks, on="AlbumId")

customer_album = (
    merged.groupby(['CustomerId', 'AlbumId'])
    .agg(PurchasedTracks=('TrackId', 'nunique'), TotalTracks=('TotalTracks', 'first'))
    .reset_index()
)

customer_album['FullAlbum'] = customer_album['PurchasedTracks'] >= customer_album['TotalTracks']

customer_preferences = (
    customer_album.groupby('CustomerId')['FullAlbum']
    .any()
    .reset_index()
)

customer_preferences['Preference'] = customer_preferences['FullAlbum'].apply(
    lambda x: 'Full Album' if x else 'Individual Tracks'
)

preference_summary = (
    customer_preferences['Preference'].value_counts(normalize=True) * 100
).reset_index()
preference_summary.columns = ['Preference', 'Percentage']

print(preference_summary)

          Preference  Percentage
0  Individual Tracks    64.40678
1         Full Album    35.59322


In [8]:
#5. A customer is considered to prefer individual tracks if they have purchased only a subset of tracks from an album.
invoice_tracks_query = """
SELECT 
    i.CustomerId,
    il.TrackId,
    t.AlbumId
FROM 
    invoice_items il
JOIN 
    invoices i ON il.InvoiceId = i.InvoiceId
JOIN 
    tracks t ON il.TrackId = t.TrackId;
"""
invoice_tracks = pd.read_sql_query(invoice_tracks_query, conn)

album_tracks_query = """
SELECT 
    AlbumId,
    COUNT(*) AS TotalTracks
FROM 
    tracks
GROUP BY 
    AlbumId;
"""
album_tracks = pd.read_sql_query(album_tracks_query, conn)

merged = invoice_tracks.merge(album_tracks, on='AlbumId')

customer_album_tracks = (
    merged.groupby(['CustomerId', 'AlbumId'])
    .agg(
        PurchasedTracks=('TrackId', 'nunique'),
        TotalTracks=('TotalTracks', 'first')
    )
    .reset_index()
)

customer_album_tracks['IsFullAlbum'] = customer_album_tracks['PurchasedTracks'] >= customer_album_tracks['TotalTracks']

customer_preferences = (
    customer_album_tracks.groupby('CustomerId')['IsFullAlbum']
    .any()
    .reset_index()
)

customer_preferences['Preference'] = customer_preferences['IsFullAlbum'].apply(
    lambda x: 'Full Album' if x else 'Individual Tracks'
)

preference_summary = (
    customer_preferences['Preference'].value_counts(normalize=True) * 100
).reset_index()
preference_summary.columns = ['Preference', 'Percentage']

print(preference_summary)

          Preference  Percentage
0  Individual Tracks    64.40678
1         Full Album    35.59322


In [1]:
#6. Provide a summary of the percentage of customers who fall into each category (individual tracks vs. full albums).
import sqlite3
import pandas as pd

conn = sqlite3.connect('chinook.db')

invoice_items = pd.read_sql_query("SELECT InvoiceId, TrackId FROM invoice_items", conn)
tracks = pd.read_sql_query("SELECT TrackId, AlbumId FROM tracks", conn)
invoices = pd.read_sql_query("SELECT InvoiceId, CustomerId FROM invoices", conn)

invoice_data = invoice_items.merge(tracks, on='TrackId').merge(invoices, on='InvoiceId')

album_track_counts = tracks.groupby('AlbumId').size().reset_index(name='TotalTracks')

invoice_album_counts = invoice_data.groupby(['InvoiceId', 'CustomerId', 'AlbumId']).size().reset_index(name='TracksPurchased')

invoice_album_counts = invoice_album_counts.merge(album_track_counts, on='AlbumId')

invoice_album_counts['IsFullAlbum'] = invoice_album_counts['TracksPurchased'] == invoice_album_counts['TotalTracks']

customer_behavior = invoice_album_counts.groupby('CustomerId')['IsFullAlbum'].agg(
    AnyFullAlbum='any',
    OnlyFullAlbums='all'
).reset_index()

def classify(row):
    if row['OnlyFullAlbums']:
        return 'Full Albums Only'
    elif row['AnyFullAlbum']:
        return 'Mixed (Albums + Tracks)'
    else:
        return 'Individual Tracks Only'

customer_behavior['PurchaseType'] = customer_behavior.apply(classify, axis=1)

summary = customer_behavior['PurchaseType'].value_counts(normalize=True).mul(100).round(2).reset_index()
summary.columns = ['PurchaseType', 'Percentage']

print(summary)


              PurchaseType  Percentage
0   Individual Tracks Only       64.41
1  Mixed (Albums + Tracks)       35.59
